<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/channel_steepness_and_concavity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing channel steepness and the concavity index with lsdtt-chi-mapping

`lsdtt-chi-mapping` is one of the command line tools included in *LSDTopoTools*. This part of *LSDTopoTools* contains many routines for looking at channel steepness indices as well as concavity indices. 

If you want more background you can read these two papers:

* Mudd, S.M., Attal, M., Milodowski, D.T., Grieve, S.W.D., Valters, D.A., 2014. A statistical framework to quantify spatial variation in channel gradients using the integral method of channel profile analysis. Journal of Geophysical Research: Earth Surface 119, 138–152. https://doi.org/10.1002/2013JF002981

* Mudd, S.M., Clubb, F.J., Gailleton, B., Hurst, M.D., 2018. How concave are river channels? Earth Surface Dynamics 6, 505–523. https://doi.org/10.5194/esurf-6-505-2018


## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

First we install `lsdviztools`. 

Note that if we install `condacolab` first the `lsdviztools` installation fails. 

In [ ]:
!pip install lsdviztools

Now we need to install lsdtopotools. We do this using something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now install `lsdtopotools`. The `&> /dev/null` bit just stops a bunch of rubbish being printed to the screen.

In [ ]:
!mamba install lsdtopotools &> /dev/null

## Download some data

We need to get some data to download. 

For this example we will work nean Xi'an, China (西安) that we studied in this paper:

* Wang, Y.Z., Mudd, S.M., 2021. Evidence for and against landscape transience in the Northern Qinling Mountains, China. Geomorphology 391, 107890. https://doi.org/10.1016/j.geomorph.2021.107890

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key. 

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Xian"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [33.672715066202954, 107.54480443333152], 
                        upper_right_coordinates = [34.16323953210814, 109.4363649228437],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

We can check to see if the file has downloaded

In [ ]:
!ls

## A simple hillshade image

We can run `lsdtt-chi-mapping` with a little interface in `lsdviztools` called the `lsdtt_driver`. Here we are only going to write a hillshade raster.  

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can plot the image using a `lsdviztools` function. I am printing this to a file and then using the image display option since it allows us to make the figure a bit bigger and in addition you can keep the png image for other uses later. 

You can save the figure as well. This allows you to show it in this notebook in a larger format:

In [ ]:
%%capture   
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code
# WARNING %%capture must be the first line in a code block (you can't even have comments above)
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

# Selecting basins

## Automated basin selection

*LSDTopoTools* automates selection of basins. The default way it does this is to grab basins within a window of contributing sizes. Here is an example:

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "minimum_basin_size_pixels" : "10000",
                    "maximum_basin_size_pixels" : "5000000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%%capture             
Base_file = r_prefix
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

In this case algorithmic identification is a bit messy for various reasons. This place has a big floodplain near XI'an, that we want to avoid. We also want to avoid all the little basins draining to the south. 

## Choosing basin outlets

One way to get all the basins we want it to make a file with the latitude and longitude of the outlets. We can go into google maps and right click on locations, and then copy these into a latitude and longitude list.

In [ ]:
# Import pandas library
import pandas as pd

data = [ [34.05564682001868, 108.20774244976475],
         [34.023214359957834, 108.5345857094824],
         [34.17775506582951, 107.66257870033067],
         [34.05259180332853, 108.33601473723209],
         [33.996904264107016, 108.75290368815703],
         [34.0171585070696, 108.96363877748513]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

We can use the linux `cat` command to make sure the file is what we expect.

In [ ]:
!cat basin_outlets.csv

## Get the basins

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()


In [ ]:
%%capture             
Base_file = r_prefix
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

# Looking at concavity

Let's now run an analysis to test the concavity index. This will take a little while (>20 minutes). You should go and get a coffee or read a book now. 

In [ ]:
lsdtt_parameters = {"estimate_best_fit_movern_no_bootstrap" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls